In [1]:
import urllib
import json
import pandas as pd
import time

In [2]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [3]:
# Format a request URI for the Genius API
search_term = 'Ayla Celik'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.parse.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [26]:
def get_songs():
    try:
        annotations = pd.read_csv('annotations.tsv', names=['song_id','lyric','annotation'], sep='\t')
    except:
        return pd.read_csv('song_ids_2.csv', names=['artist','song'], sep=',')
    songs = pd.read_csv('song_ids_2.csv', names=['artist','song'], sep=',')
    
    print("SONGS:")
    print(songs.dtypes())
    print(songs.head())
    print("ANNOTATIONS:")
    print(annotations.dtypes())
    print(annotations.head())
    
    
    last_id = str(annotations.song_id.values[-1])
    print(last_id)
    print(type(last_id))
    i = songs[songs.song == last_id].index[0]
    return songs[i+1:]

In [27]:
# Function to recursively search the annotation dict
# and get the text, skipping blockquotes, formatting, images, etc.

def recurse(children):
    st = ''
    for child in children:
        #print(child)
        if type(child) == dict and 'tag' in child:
            if child['tag'] == 'p' or child['tag'] == 'a':
                st += recurse(child['children'])
            elif child['tag'] == 'blockquote':
                st += '[BLOCKQUOTE]'
        elif type(child) == list:
            st += recurse(child)
        elif type(child) == str:
            st += child + ' '
    return st

In [28]:
def append_annotations(songdf, batchsize):
    # Get a chunk of song ids
    song_ids = songdf.head(batchsize).song.astype(int)

    with open('annotations.tsv', 'a') as f:

        for song_id in song_ids:

            # Format the search query using the 'referents' api
            # Given a song id, returns all of its referents (lines that have been annotated)
            # and the corresponding annotations.
            querystring = "https://api.genius.com/referents?song_id=" + str(song_id) + "&per_page=50"
            request = urllib.request.Request(querystring)
            request.add_header("Authorization", "Bearer " + token)
            request.add_header("User-Agent", "")

            # Do internet stuff
            # Send the request to Genius, and parse the response
            try:
                response = urllib.request.urlopen(request, timeout=10)
                string = response.read().decode('utf-8')
                json_obj = json.loads(string)
            except:
                print('***Failed on song id: ' + str(song_id))
                continue

            # The data we want can be found here
            # For this song there are 9 lyric/annotation pairs
            referents = json_obj['response']['referents']

            # Many songs return zero annotations. If we get annotations, parse through them
            if len(referents) > 0:
                for ref in referents:
                    lyric = ref['fragment'].replace('\n', ' ')
                    try:
                        annotation = recurse(ref['annotations'][0]['body']['dom']['children']).replace('\n', ' ')
                    except:
                        print('***Empty annotation on song id: ' + str(song_id))
                        continue
                    
                    print(lyric)
                    f.write(str(song_id) + '\t' + lyric + '\t' + annotation + '\n')
                    
            time.sleep(0.5)

In [29]:
songdf = get_songs()
append_annotations(songdf, 200)

SONGS:


TypeError: 'Series' object is not callable